2_preprocessing.ipynb

공공데이터포털에서 대구광역시_나드리콜 접수 데이터 내려받고 전처리 (csv 파일로 내보내기)

In [1]:
import pydeck
import pandas as pd
import datetime
pydeck.__version__, pd.__version__

('0.7.1', '1.4.1')

In [2]:
df = pd.read_csv('data/data.csv')
df.head()

,접수시간,목적지분류,질환분류,장애정도,휠체어 유무,출발지 위도,출발지 경도,목적지 위도,목적지 경도,배차시간(초),고객 승차시간,고객 하차시간,출발지 행정동,목적지 행정동,배차구분,배차 차량종류,배차 차량번호
0,2017-01-01 00:05:41,종교시설,지체,중증,무,35.867470,128.585647,35.871683,128.580994,4005.0,2017-01-01 01:12:27,2017-01-01 01:18:58,성내2동,대신동,배차,특장차량,NCID0001
1,2017-01-01 00:20:14,집,뇌병변,중증,무,35.870856,128.580534,35.912621,128.654441,280.0,2017-01-01 00:24:55,2017-01-01 00:46:18,대신동,도평동,배차,개인택시,NCID0002
2,2017-01-01 00:24:08,집,지체,중증,무,35.865308,128.594390,35.899288,128.596239,2504.0,2017-01-01 01:05:53,2017-01-01 01:18:23,남산1동,산격1동,배차,개인택시,NCID0003
3,2017-01-01 00:26:37,의료시설,지적,중증,무,35.841549,128.562663,35.855213,128.551146,423.0,2017-01-01 00:33:41,2017-01-01 00:42:43,대명10동,두류3동,배차,개인택시,NCID0003
4,2017-01-01 00:35:08,집,지체,중증,무,35.869211,128.583356,35.869404,128.582572,1025.0,2017-01-01 00:52:14,2017-01-01 00:52:16,성내2동,범물1동,배차,개인택시,NCID0004


In [3]:
# rename
df = df.rename(columns={'출발지 위도': 'start_lat', '출발지 경도': 'start_lng', '목적지 위도': 'depart_lat', '목적지 경도': 'depart_lng', '고객 승차시간': 'start_time', '고객 하차시간': 'depart_time'})

# convert datetime to epoch time
df['start_time'] = df['start_time'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').timestamp())
df['depart_time'] = df['depart_time'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').timestamp())

df.head()

,접수시간,목적지분류,질환분류,장애정도,휠체어 유무,start_lat,start_lng,depart_lat,depart_lng,배차시간(초),start_time,depart_time,출발지 행정동,목적지 행정동,배차구분,배차 차량종류,배차 차량번호
0,2017-01-01 00:05:41,종교시설,지체,중증,무,35.867470,128.585647,35.871683,128.580994,4005.0,1.483201e+09,1.483201e+09,성내2동,대신동,배차,특장차량,NCID0001
1,2017-01-01 00:20:14,집,뇌병변,중증,무,35.870856,128.580534,35.912621,128.654441,280.0,1.483198e+09,1.483199e+09,대신동,도평동,배차,개인택시,NCID0002
2,2017-01-01 00:24:08,집,지체,중증,무,35.865308,128.594390,35.899288,128.596239,2504.0,1.483200e+09,1.483201e+09,남산1동,산격1동,배차,개인택시,NCID0003
3,2017-01-01 00:26:37,의료시설,지적,중증,무,35.841549,128.562663,35.855213,128.551146,423.0,1.483198e+09,1.483199e+09,대명10동,두류3동,배차,개인택시,NCID0003
4,2017-01-01 00:35:08,집,지체,중증,무,35.869211,128.583356,35.869404,128.582572,1025.0,1.483200e+09,1.483200e+09,성내2동,범물1동,배차,개인택시,NCID0004


data format
```
[
    {
        'id': 1,
        'path': [
            [111, 222],
            [111, 222],
            [111, 222],
        ],
        'timstamp': [1,2,3],
    }, ..
]
```

In [4]:
df_new = []

MIN_TIME = df['start_time'].min()

for i in range(len(df)):
    more = {
        'id': df['배차 차량번호'][i],
        'path': [
            [df['start_lng'][i], df['start_lat'][i]],
            [df['depart_lng'][i], df['depart_lat'][i]],
        ],
        'timestamp': [df['start_time'][i] - MIN_TIME, df['depart_time'][i] - MIN_TIME],
    }
    df_new.append(more)

df_new = pd.DataFrame(df_new)

print(len(df_new))
df_new.head()

12


,id,path,timestamp
0,NCID0001,"[[128.58564680848303, 35.86746958399148], [128...","[2852.0, 3243.0]"
1,NCID0002,"[[128.58053355840923, 35.870856397223186], [12...","[0.0, 1283.0]"
2,NCID0003,"[[128.594389611206, 35.86530843553168], [128.5...","[2458.0, 3208.0]"
3,NCID0003,"[[128.56266323989422, 35.841549380508866], [12...","[526.0, 1068.0]"
4,NCID0004,"[[128.5833563575808, 35.86921129530684], [128....","[1639.0, 1641.0]"


In [5]:
# 생성한 osrm_api가 잘 동작하는지 테스트
from osrm_api import get_route

print('전체 데이터 수:', len(df_new['path']))

print('출발지 좌표:', df_new['path'][0][0])
print('도착지 좌표:', df_new['path'][0][1])

df_route = get_route(df_new['path'][0], start_time=df_new['timestamp'][0][0])

print('0번 데이터의 waypoint 개수', df_route['path'][0].__len__())
df_route

전체 데이터 수: 12
출발지 좌표: [128.58564680848303, 35.86746958399148]
도착지 좌표: [128.58099363496564, 35.87168338504525]
0번 데이터의 waypoint 개수 5


,path,timestamp
0,"[[128.586488, 35.867508], [128.586577, 35.8682...","[2864.7, 2891.0, 2913.0, 2932.4, 2932.4]"


In [6]:
from osrm_api import get_route

# osrm으로 waypoint를 얻어냄
for i in range(len(df_new)):
    if i % len(df_new)//10 == 0:
        print('=', end='', flush=True)
    
    df_route = get_route(df_new['path'][i], start_time=df_new['timestamp'][i][0])
    df_new['path'][i] = df_route['path'][0]
    df_new['timestamp'][i] = df_route['timestamp'][0]

df_new

,id,path,timestamp
0,NCID0001,"[[128.586488, 35.867508], [128.586577, 35.8682...","[2864.7, 2891.0, 2913.0, 2932.4, 2932.4]"
1,NCID0002,"[[128.580994, 35.870917], [128.581082, 35.8704...","[11.8, 134.9, 159.4, 186.8, 194.3, 235.4, 345...."
2,NCID0003,"[[128.594369, 35.865235], [128.599484, 35.8637...","[2492.2, 2570.1, 2610.7, 2713.8999999999996, 2..."
3,NCID0003,"[[128.562665, 35.84149], [128.560924, 35.84198...","[562.8, 577.5999999999999, 584.3, 631.3, 656.0..."
4,NCID0004,"[[128.583466, 35.869256], [128.582473, 35.8694...","[1691.9, 1691.9]"
5,NCID0005,"[[128.554118, 35.80951], [128.554429, 35.80839...","[1636.4, 1652.3000000000002, 1762.9, 1844.8000..."
6,NCID0006,"[[128.497841, 35.854814], [128.498488, 35.8548...","[1305.7, 1318.8, 1370.8999999999999, 1423.6, 1..."
7,NCID0007,"[[128.6287, 35.877129], [128.617632, 35.87459]...","[2036.5, 2088.7, 2155.6, 2165.4, 2217.9, 2217.9]"
8,NCID0008,"[[128.530677, 35.838177], [128.530679, 35.8372...","[1890.7, 1894.6000000000001, 1899.600000000000..."
9,NCID0001,"[[128.595459, 35.869854], [128.59722, 35.86997...","[3466.4, 3482.7000000000003, 3537.3, 3537.3]"


In [7]:
df_new.to_csv('data/data_osrm.csv', index=False)